## Network from Kell, Yamins, Shook, Norman-Haignere, McDermott, 2018

This notebook shows how to create a tensorflow graph for the network with the weights and biases used in <a href="https://www.cell.com/neuron/fulltext/S0896-6273(18)30250-2">Kell et al., 2018</a>. This notebook also gives an example of how to pass a sound into the network.


### Note on network input

The input to the network is a "cochleagram", a time-frequency decomposition of a sound that is similar to a spectrogram. Below we provide examples of how to pass a pre-computed cochleagram into the network, as well as how to compute the cochleagram for an example wav and then pass that cochleagram to the network.

### Dependencies

Most of the dependencies to run this are relatively standard. However, please note the following:
- This notebook was tested and run with version 1.5.0 of `tensorflow`. It was not tested with other versions.
- `pycochleagram` is a module to generate cochleagrams to pass sounds into the network, which can be found <a href="https://github.com/mcdermottLab/pycochleagram">here</a>.
- `PIL` is the Python Image Library.

### Contact
If you have any questions, please contact Alex Kell. Email: < first_name >< last_name >@mit.edu.

Thanks, and enjoy!

In [56]:
import IPython.display as ipd
import sys
sys.path.append('./network/')
from branched_network_class import branched_network
import tensorflow as tf
import scipy.io.wavfile as wav
import matplotlib as plt 
%pylab inline
import librosa

Populating the interactive namespace from numpy and matplotlib


In [66]:
# import the following to run demo_from_wav()
import pyco.cochleagram as cgram
from PIL import Image

In [58]:
## Some helper functions
def resample(example, new_size):
    im = Image.fromarray(example)
    resized_image = im.resize(new_size, resample=Image.ANTIALIAS)
    return np.array(resized_image)

def plot_cochleagram(cochleagram, title): 
    plt.figure(figsize=(6,3))
    plt.matshow(cochleagram.reshape(256,256), origin='lower',cmap=plt.cm.Blues, fignum=False, aspect='auto')
    plt.yticks([]); plt.xticks([]); plt.title(title); 
    
def play_wav(wav_f, sr, title):   
    print title+':'
    ipd.display(ipd.Audio(wav_f, rate=sr))

In [1]:
def demo_from_wav(filename):
    tf.reset_default_graph()
    net_object = branched_network()
    word_key = np.load('./demo_stim/logits_to_word_key.npy') # load logits to word key
    music_key = np.load('./demo_stim/logits_to_genre_key.npy') # load logits to word key 
    
    
    # generate cochleagram, then pass cochleagram through network and get logits for word branch
    
    c_gram = generate_cochleagram(filename)
    logits = net_object.session.run(net_object.word_logits, feed_dict={net_object.x: c_gram})
    prediction = word_key[np.argmax(logits, axis=1)]
    print "Speech Example ... \n clean speech, actual label: Cheese, predicted_label: " \
        + prediction[0] +'\n'
    
demo_from_wav('./data/cantonese/cantonese1/also.wav')

NameError: global name 'tf' is not defined

In [54]:
word_key = np.load('./demo_stim/logits_to_word_key.npy') # load logits to word key
allowed_words = [
    'also',
    'call',
    'cheese',
    'fresh',
    'into',
    'need',
    'small',
    'store',
    'these',
    'things',
]

for i, word in enumerate(word_key):
    if word in allowed_words:
        print i, word

28 small
78 call
139 things
219 cheese
241 store
263 need
396 these
397 also
514 into
585 fresh


In [69]:
def generate_cochleagram(filename):
    # define parameters
    wav_f, sr = librosa.core.load(filename, sr=16000) # note the sampling rate is 16000hz.
    n = 50
    low_lim, hi_lim = 20, 8000
    sample_factor, pad_factor, downsample = 4, 2, 200
    nonlinearity, fft_mode, ret_mode = 'power', 'auto', 'envs'
    strict = True
    # create cochleagram
    c_gram = cgram.cochleagram(wav_f, sr, n, low_lim, hi_lim, 
                               sample_factor, pad_factor, downsample,
                               nonlinearity, fft_mode, ret_mode, strict)

    # rescale to [0,255]
    c_gram_rescaled =  255*(1-((np.max(c_gram)-c_gram)/np.ptp(c_gram)))
    
    # reshape to (256,256)
    c_gram_reshape_1 = np.reshape(c_gram_rescaled, (211,400))
    c_gram_reshape_2 = resample(c_gram_reshape_1,(256,256))

    # prepare to run through network -- i.e., flatten it
    c_gram_flatten = np.reshape(c_gram_reshape_2, (1, 256*256)) 

    return c_gram_flatten